In [5]:
from papago_trans import *
import time
from selenium import webdriver
import tika
from tika import parser
import datetime
import re
file_name = input("Input file name you would like to translate (ex. my_file): ")

Input file name you would like to translate (ex. my_file): 법정동 리스트


### Read text from PDF

In [19]:
article = get_article(file_name)

In [9]:
import pandas as pd
with open("법정동 리스트.txt", 'r', encoding='cp949') as f:
    content = f.readlines()
## split
content = [_.split('\t') for _ in content]
## get rid of \n
content = [[i,j,z.replace('\n', '')] for i,j,z in content]

## make into data frame
df = pd.DataFrame(content[1:], columns = content[0])

In [10]:
d = list()
for i in df.법정동코드:
    if '00000000' in i:
        d.append(i)

In [12]:
busan = df[(df.법정동명.str.contains('부산광역시')) & (df.폐지여부!='폐지')]
busan = busan[~busan.법정동코드.str.contains('000000')]

In [14]:
busan_codes = busan.법정동코드.values
busan_kor = busan.법정동명.values

busan['short'] = busan.법정동코드.str.slice(stop=5)

busan_s_codes = busan.drop_duplicates(subset=['short']).short.values
busan_s_kor = ['부산광역시 중구', '부산광역시 서구', '부산광역시 동구', '부산광역시 영도구', '부산광역시 부산진구',
               '부산광역시 동래구', '부산광역시 남구', '부산광역시 북구', '부산광역시 해운대구', '부산광역시 사하구',
               '부산광역시 금정구', '부산광역시 강서구', '부산광역시 연제구', '부산광역시 수영구',
               '부산광역시 사상구', '부산광역시 기장군']
len(busan_s_codes)==len(busan_s_kor)

In [49]:
seoul = df[(df.법정동명.str.contains('서울')) & (df.폐지여부!='폐지')]
seoul = seoul[~seoul.법정동코드.str.contains('000000')]

seoul_codes = seoul.법정동코드.values
seoul_kor = seoul.법정동명.values

seoul['short'] = seoul.법정동코드.str.slice(stop=5)
seoul_s_codes = seoul.drop_duplicates(subset=['short']).short.values
seoul_s_kor = ['서울특별시 종로구', '서울특별시 중구', '서울특별시 용산구', '서울특별시 성동구', '서울특별시 광진구',
               '서울특별시 동대문구', '서울특별시 중랑구', '서울특별시 성북구', '서울특별시 강북구', '서울특별시 도봉구',
               '서울특별시 노원구', '서울특별시 은평구', '서울특별시 서대문구', '서울특별시 마포구',
               '서울특별시 양천구', '서울특별시 강서구', '서울특별시 구로구', '서울특별시 금천구',
               '서울특별시 영등포구', '서울특별시 동작구', '서울특별시 관악구', '서울특별시 서초구', 
               '서울특별시 강남구', '서울특별시 송파구', '서울특별시 강동구']
len(seoul_s_codes)==len(seoul_s_kor)

True

### Preprocess text!!!!! (Done manually)

```Python
# some historical examples

# all the things including, and between
re.sub('\n\n[0-9]* Jacquelynne [A-Z.\\\ a-z\n©0-9,:;()-/]*s1096934\n\n\n\n',
       '', article)

article.replace('-\n', '')

article.replace('\n', ' ')
```

In [20]:
# preprocess here
article = article.replace('-\n', '')
article = re.sub('Comparative Education Review [0-9]*', '', article)
article = article.replace('BAKER ET AL', '')
article = article.replace('ANALYSIS OF THE “HEYNEMAN-LOXLEY EFFECT” ON MATHEMATICS AND SCHOOL ACHIEVEMENT', '')
article = re.sub('\n+', '\n', article)
article = article.replace('\n', ' ')

### Make article to list, and send them to papago

In [28]:
start_driver()

In [14]:
papa_list = article2lists(article, delim='\n')

Good to go


In [29]:
busan_eng = papa_translate(busan_kor)
busan_s_eng = papa_translate(busan_s_kor)

Start translating
Errors occurred: []
Start translating
Errors occurred: []


In [35]:
seoul_eng = papa_translate(seoul_kor)
seoul_s_eng = papa_translate(seoul_s_kor)

Start translating
Errors occurred: []
Start translating
Errors occurred: []


In [41]:
def only_kor(translated_list):
    translated_list = [_.split('\n')[:-1] for _ in translated_list]
    translated_list = [item for sublist in translated_list for item in sublist]
    return translated_list

In [42]:
busan_eng = only_kor(busan_eng)
busan_s_eng = only_kor(busan_s_eng)
seoul_eng = only_kor(seoul_eng)
seoul_s_eng = only_kor(seoul_s_eng)

In [51]:
trans_busan_fullPNU = pd.DataFrame(data=list(zip(busan_codes, busan_eng, busan_kor)))
trans_busan_shortPNU = pd.DataFrame(data=list(zip(busan_s_codes, busan_s_eng, busan_s_kor)))

In [54]:
trans_seoul_fullPNU = pd.DataFrame(data=list(zip(seoul_codes, seoul_eng, seoul_kor)))
trans_seoul_shortPNU = pd.DataFrame(data=list(zip(seoul_s_codes, seoul_s_eng, seoul_s_kor)))

In [59]:
trans_busan_fullPNU.to_excel('trans_busan_fullPNU.xlsx')
trans_busan_shortPNU.to_excel('trans_busan_shortPNU.xlsx')
trans_seoul_fullPNU.to_excel('trans_seoul_fullPNU.xlsx')
trans_seoul_shortPNU.to_excel('trans_seoul_shortPNU.xlsx')

In [27]:
papa_save(flat_list, file_name='6.Miller_trans.txt')
# papa_fix_saved('6.Miller_trans.txt')

Done. The file name is: 6.Miller_trans.txt


In [36]:
end_driver()